In [2]:
import findspark

In [3]:
findspark.init('/home/adeola/spark-2.4.2-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('clusters').getOrCreate()

In [6]:
dataset = spark.read.format('libsvm').load('sample_kmeans_data.txt')

In [7]:
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [8]:
#since we are working with unsupervised learning, we will use only the feature column
final_data = dataset.select('features')

In [9]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [10]:
from pyspark.ml.clustering import KMeans

In [11]:
#create your kmeans model with setk(2)and setSeed(1). SetK is the number of clusters you will have
kmeans = KMeans().setK(2).setSeed(1)

In [12]:
#Now fit the final_data into kmeans to create a model
model = kmeans.fit(final_data)

In [14]:
#now check within set sum of squared errors wssse by using computeCost() of the final_data
wssse = model.computeCost(final_data)

In [15]:
print (wssse) #note that as k increases, this value decreases

0.11999999999994547


In [16]:
#to calculate the centers of the model
centers = model.clusterCenters()

In [17]:
centers

[array([0.1, 0.1, 0.1]), array([9.1, 9.1, 9.1])]

In [18]:
#now let us transfrom the final_data
results = model.transform(final_data)

In [19]:
results.show() #this will group the features into 2 under prediction, 0 group and 1 group

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         0|
|(3,[0,1,2],[0.1,0...|         0|
|(3,[0,1,2],[0.2,0...|         0|
|(3,[0,1,2],[9.0,9...|         1|
|(3,[0,1,2],[9.1,9...|         1|
|(3,[0,1,2],[9.2,9...|         1|
+--------------------+----------+



In [24]:
#if we increase SetK to 3
kmeans1 = KMeans().setK(3).setSeed(1)

In [25]:
model1 = kmeans1.fit(final_data)

In [26]:
wssse1 = model.computeCost(final_data)

In [27]:
print(wssse1)

0.07499999999994544


In [28]:
center2 = model1.clusterCenters()

In [29]:
center2

[array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [31]:
results_new = model1.transform(final_data)

In [32]:
results_new.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

